In [5]:
using OhMyJulia
using DataFrames
using PyCall
using NaturalSort
@pyimport gurobipy as grb

include("path.jl")
include("weight.jl")
ENV["PATH"] *= ":/usr/local/gurobi/bin/:/home/zhangshiwei/.opam/4.06.0/bin"

#data_list = ["abilene", "gscale", "Cernet", "Cesnet201006", "Chinanet", "Esnet", "Garr200112", "Grnet", "Ibm", "jelly16", "Nordu1997", "Sprint", "Tinet"]
data_list = ["Cernet" , "Globalcenter" , "Goodnet" , "Gridnet" , "gscale" , "Janetbackbone" ,
"Rediris" , "Renater2010" , "Sinet" , "Sprint" , "SprintINET" , "Surfnet" ,
"SwitchL3" , "TataNld" , "Tinet" , "Uninett2011" , "Uunet" , "VtlWavenet2011" , "Xeex" , "Xspedius"]
algo_list = ["Ecmp", "Edksp", "Ksp", "Mcf", "Raeke", "Vlb"]

6-element Array{String,1}:
 "Ecmp" 
 "Edksp"
 "Ksp"  
 "Mcf"  
 "Raeke"
 "Vlb"  

## gen demand and budget

In [7]:
function gen_gravity(data)
    hosts = readlines("data/$data.hosts")
    open("data/$data.demands", "w") do fout
        for k in 1:6
            #α = 2^(23 + k)
            α = 2^(23 + k)
            weigths = map(x->rand(), hosts)
            list = [ α * weigths[i] * weigths[j]
                     for i in 1:length(hosts)
                     for j in 1:length(hosts) ]
            println(fout, join(list, ' '))
        end
    end
end
for i in data_list
    gen_gravity(i)
end

In [9]:
function gen_budgets(data)
    hosts = readlines("data/$data.hosts")
    n = length(hosts)
    open("data/$data.budgets", "w") do fout
        for h in hosts
            println(fout, 2 * (n * n - n))
        end
    end
end
for i in data_list
    gen_budgets(i)
end

In [10]:
function read_topo(file)
    nodes, edges = Set{String}(), String[]
    for line in eachline(file) @when (m = match(r"(s\d+) -> (s\d+)", line)) != nothing
        push!(nodes, m[1], m[2])
        push!(edges, "$(m[1]) $(m[2])")
    end
    nodes, edges
end

function read_demand(hosts, demands)
    results = []
    hosts = readlines(hosts)
    n = length(hosts)
    for line in eachline(demands)
        line = map(x->parse(f64, x), split(line, ' '))
        demand = Dict{String, Float64}()
        for i in 1:n, j in 1:n @when i != j
            demand["$(hosts[i]) $(hosts[j])"] = line[(i-1) * n + j] / 2^20 # to Megabyte
        end
        push!(results, demand)
    end
    results
end

function read_budget(hosts, budgets)
    Dict(zip(map(x->replace(x, 'h', 's'), readlines(hosts)),
             map(x->parse(Int, x),        readlines(budgets))))
end

function write_scheme(io, scheme)
    for (pair, paths) in scheme
        println(io, pair, ':')
        for (path, w) in paths
            p = map(x->split(x, ' '), path)
            println(io, join(map(car, p) ++ cadr(p[end]), " -> "), " @ ", w)
        end
        println(io)
    end
end

write_scheme (generic function with 1 method)

In [20]:
for data in ("Cernet",) #data_list
    io, process = open(`yates -budget 16 $data`)
    path_sets = parse_schemes(io)
    nodes, edges = read_topo("data/$data.dot")
    demands = read_demand("data/$data.hosts", "data/$data.demands")
    budgets = read_budget("data/$data.hosts", "data/$data.budgets")
    
    for (i, demand) in enumerate(demands), algo in algo_list
        for (name, select) in (("program", select_program),
                               # ("greedy", select_greedy),
                               ("hardnop", select_hard_nop))
            open("results/$data-$i-$algo-$name.result", "w") do fout
                tic()
                raw_scheme = select(nodes, edges, path_sets[algo], budgets)
                time1 = toq()
                tic()
                m1_scheme, Z = minimize_maximum_link_utilization(nodes, edges, demand, raw_scheme)
                time2 = toq()
                tic()
                m2_scheme, Zs = minimize_maximum_link_utilization_then_maximize_throuput(nodes, edges, demand, raw_scheme)
                time3 = toq()
                
                println(fout, "number of nodes (switch): ", length(nodes))
                println(fout, "number of edges (core): ", length(edges))
                println(fout, "number of demand pairs: ", length(demand))
                println(fout, "average budgets: ", mean(values(budgets)))
                println(fout)
                
                println(fout, "time elapsed during path selection: ", time, 's')
                
                println(fout, "Z: ", Z)
                println(fout, "total throuput before the second step:", sum(values(demand)) / max(Z, 1))
                println(fout, "total throuput after the second step:", sum(d / Zs[pair] for (pair, d) in demand))
                println(fout)
                
                println(fout, "number of path through each node:")
                node_dict = Dict(n => 0 for n in nodes)
                for (pair, paths) in m2_scheme, (path, weight) in paths, edge in path
                    n = split(edge, ' ') |> car
                    if car(n) == 's'
                        node_dict[n] += 1
                    end
                end
                for (node, number) in sort(collect(node_dict), lt=natural, by=car)
                    println(fout, "  ", node, ": ", number)
                end
                println(fout, "  average: ", sum(values(node_dict)) / length(node_dict))
                println(fout)
                
                println(fout, "number of path, total flow, and link utiliaztion of each edge:")
                edge_dict = Dict(e => f64[] for e in edges)
                for (pair, paths) in m2_scheme, (path, weight) in paths, edge in path @when 'h' ∉ edge
                    push!(edge_dict[edge], demand[pair] * weight)
                end
                for (edge, flows) in sort(collect(edge_dict), lt=natural, by=car)
                    println(fout, "  ", edge, ": ", length(flows), ", ", sum(flows), ", ", sum(flows) / 100000, '%')
                end
                println(fout, "  average: ", sum(length(v) for v in values(edge_dict)) / length(edge_dict), 
                                       ", ", sum(sum(v) for v in values(edge_dict)) / length(edge_dict),
                                       ", ", sum(sum(v) for v in values(edge_dict)) / length(edge_dict) / 100000, '%')
                println(fout)
                
                println(fout, "number of path and demand ratio of each st pair:")
                pair_dict = Dict(pair => length(paths) for (pair, paths) in m2_scheme)
                for (pair, nflows) in sort(collect(pair_dict), lt=natural, by=car)
                    println(fout, "  ", pair, ": ", nflows, ", ", 1 / Zs[pair])
                end
                println(fout, "  average: ", sum(values(pair_dict)) / length(pair_dict),
                                       ", ", sum(d / Zs[pair] for (pair, d) in demand) / sum(values(demand)))
                println(fout)
                
                println(fout, "path set:")
                write_scheme(fout, path_sets[algo])
                println(fout)
                
                println(fout, "paths selected:")
                write_scheme(fout, raw_scheme)
                println(fout)
                
                println(fout, "paths after the first step (minimize maximum link utilization):")
                write_scheme(fout, m1_scheme)
                println(fout)
                
                println(fout, "paths after the second step (maximize total throuput):")
                write_scheme(fout, m2_scheme)
            end
        end
    end
end

LoadError: [91mMethodError: no method matching zero(::Type{Any})[0m
Closest candidates are:
  zero(::Type{Union{Missings.Missing, T}}) where T at /home/zhangshiwei/.julia/v0.6/Missings/src/Missings.jl:97
  zero([91m::Type{Base.LibGit2.GitHash}[39m) at libgit2/oid.jl:106
  zero([91m::Type{Base.Pkg.Resolve.VersionWeights.VWPreBuildItem}[39m) at pkg/resolve/versionweight.jl:82
  ...[39m

In [3]:
data = "jelly16"
io, process = open(`yates -budget 16 $data`)
    path_sets = parse_schemes(io)

nodes, edges = read_topo("data/$data.dot")
    demands = read_demand("data/$data.hosts", "data/$data.demands")
    budgets = read_budget("data/$data.hosts", "data/$data.budgets")


Dict{String,Int64} with 16 entries:
  "s8"  => 240
  "s5"  => 240
  "s7"  => 240
  "s14" => 240
  "s9"  => 240
  "s10" => 240
  "s6"  => 240
  "s11" => 240
  "s15" => 240
  "s3"  => 240
  "s1"  => 240
  "s13" => 240
  "s12" => 240
  "s2"  => 240
  "s4"  => 240
  "s0"  => 240